In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

import re
import nltk
from pymystem3 import Mystem
from gensim.models import KeyedVectors

/Users/dayyass/Desktop/ml_interviews/mts/venv/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
import warnings
warnings.filterwarnings("ignore")

### load data

In [3]:
data = pd.read_csv(
    "../data/russe-wsi-kit/data/additional/active-rutenten/train.csv",
    sep="\t",
)

In [4]:
data.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,альбом,2,NaN,88-94,достаточно лишь колесиком мышки крутить вниз. ...
1,2,альбом,3,NaN,85-91,"выступал в составе команды с таким названием, ..."
2,3,альбом,2,NaN,81-87,". Работает так себе, поскольку функция заточен..."
3,4,альбом,3,NaN,84-89,одержала победу в двух из пяти номинаций: 'Луч...
4,5,альбом,3,NaN,83-88,встречи с Божественным. Вы испытаете ни с чем ...


### init model

In [5]:
stemmer = Mystem()

embedder = KeyedVectors.load_word2vec_format(
    "../modules/ruscorpora_mean_hs.model.bin.gz",
    binary=True,
)

In [6]:
stopwords = nltk.corpus.stopwords.words("russian")

### find senses

In [7]:
sense_1 = {
    word: embedder.most_similar(
        positive=[f"{word}_S"],
        topn=1,
    )[0][0] \
    for word in data['word'].unique()
}

In [8]:
sense_2 = {
    word: embedder.most_similar(
        positive=[f"{word}_S"],
        negative=[sense_1[word]],
        topn=1,
    )[0][0] \
    for word in data['word'].unique()
}

In [9]:
data['sense_1'] = data['word'].replace(sense_1)
data['sense_2'] = data['word'].replace(sense_2)

In [10]:
# check senses for different words
data.drop_duplicates(['word']).head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context,sense_1,sense_2
0,1,альбом,2,NaN,88-94,достаточно лишь колесиком мышки крутить вниз. ...,фотография_S,сборник_S
450,451,анатомия,2,NaN,106-113,способна улучшить Ваш сон и самочувствие после...,физиология_S,глузский_A
545,546,базар,1,NaN,8-12,</p><p> Базар - известный центр всех экспедици...,толкучка_S,общеиндоевропейский_A
635,636,балет,3,NaN,101-106,декабря в Новосибирском государственном академ...,оперетта_S,alisa_UNKN
729,730,беда,2,NaN,86-89,защемило в душе по поводу провала Хмары и Луцк...,несчастие_S,видать_ADV


In [11]:
def get_lemma_and_pos_tag_from_word_analysis(token_analysis):
    
    lemma = token_analysis["analysis"][0]["lex"]
    pos_tag = token_analysis["analysis"][0]["gr"].split(",")[0].split("=")[0]
    
    return lemma, pos_tag


def embed_context(
    context,
    target_word=None,
    re_pattern="[\w\-]+$",
    stopwords=stopwords,
    stemmer=stemmer,
    embedder=embedder,
):
    
    tokens = []
    
    analysis = stemmer.analyze(context)
    
    for token_analysis in analysis:
        if ("analysis" in token_analysis) and (len(token_analysis['analysis']) > 0):
            lemma, pos_tag = get_lemma_and_pos_tag_from_word_analysis(token_analysis)
            token = f"{lemma}_{pos_tag}"
        
            if re.match(re_pattern, lemma) and (lemma not in stopwords):
                tokens.append(token)
    
    cnt = 0
    embedding = np.zeros(300)
    
    for token in tokens:
        
        # skip target word
        if target_word:
            if token == f"{target_word}_S":
                continue
        
        if token in embedder:
            cnt += 1
            embedding += embedder[token]
    
    return embedding / cnt

In [12]:
data["predict_sense_id"] = data.apply(
    lambda row: 1 + np.argmax([
        cosine_similarity(
            X=[
                embed_context(
                    context=row["context"],
                    target_word=row["word"],
                )
            ],
            Y=[
                embedder[row["sense_1"]],
                embedder[row["sense_2"]],
            ])[0]
    ]),
    axis=1,
)

In [13]:
data['predict_sense_id'].value_counts()

1    2387
2    1284
Name: predict_sense_id, dtype: int64

In [14]:
data.drop(["sense_1", "sense_2"], axis=1, inplace=True)

In [15]:
data.to_csv(
    "predictions/word2vec_prediction.tsv",
    sep="\t",
    index=False,
)

### valudate

In [16]:
!python3 ../data/russe-wsi-kit/evaluate.py predictions/word2vec_prediction.tsv

word	ari	count
альбом	0.145914	450
анатомия	0.081825	95
базар	0.099206	90
балет	0.008901	94
беда	-0.014507	93
бездна	-0.015271	87
билет	-0.008388	447
блок	0.038104	206
блоха	0.220007	86
брак	0.372598	96
бритва	-0.011365	85
будущее	-0.035580	83
вешалка	-0.014867	390
вилка	0.011303	302
винт	0.354304	358
галерея	0.221791	24
горбуша	-0.008748	93
горшок	0.082218	406
гроза	0.099304	95
группа	0.027579	91
	0.084359	3671
